In [13]:
from src.data_preprocessor import DataProcessor
from src.cross_validation import CrossValidation
from src.evaluation import Evaluation
from models.knn import KNN
from models.null_model import NullModelClassification, NullModelRegression
from data_configs.configs import *
import statistics

config = albalone_config
data_processor = DataProcessor(config=config)
cross_validator = CrossValidation(config=config)
classification_nullmodel = NullModelClassification(config=config)
regression_nullmodel = NullModelRegression(config=config)
knn_model = KNN(config)

In [14]:
# Data Processing

raw_data = data_processor.load_data()

data_1 = data_processor.impute_missing_values(raw_data)

data_2 = data_processor.encode_nominal_features(data_1)

data_3 = data_processor.encode_ordinal_features(data_2)

In [15]:
data_train, data_val = cross_validator.random_partition(data_3, random_state=42)

In [16]:
data_train

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Sex_F,Sex_I,Sex_M
4038,0.550,0.445,0.125,0.6720,0.2880,0.1365,0.210,11,0,1,0
1272,0.475,0.355,0.100,0.5035,0.2535,0.0910,0.140,8,0,1,0
3384,0.305,0.225,0.070,0.1485,0.0585,0.0335,0.045,7,1,0,0
3160,0.275,0.200,0.065,0.1165,0.0565,0.0130,0.035,7,0,1,0
3894,0.495,0.380,0.135,0.6295,0.2630,0.1425,0.215,12,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
3444,0.490,0.400,0.115,0.5690,0.2560,0.1325,0.145,9,1,0,0
466,0.670,0.550,0.190,1.3905,0.5425,0.3035,0.400,12,1,0,0
3092,0.510,0.395,0.125,0.5805,0.2440,0.1335,0.188,11,0,0,1
3772,0.575,0.465,0.120,1.0535,0.5160,0.2185,0.235,9,0,0,1


In [17]:
gamma = 1/(statistics.stdev(data_train[config['target_column']]))

In [18]:
results = knn_model.knn_regression(data_val, data_train, 5, gamma)['Predicted Value']
Evaluation().mean_squared_error(data_val[config['target_column']],results)


5.1315221459758895

In [12]:
hyperparameters = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for k in hyperparameters:  
    scores = []
    for i, (train_set_1, train_set_2) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, stratify=False)):
        # Train and evaluate using train_set_1
        condensed_train = knn_model.condensed_knn_regression(train_set_1,10)
        predictions_1 = knn_model.knn_regression(data_val, condensed_train, k=k, gamma=gamma)['Predicted Value']
        score = Evaluation().mean_squared_error(data_val[config['target_column']], predictions_1)
        scores.append(score)
        
    average_score = sum(scores) / len(scores)
    print(f"Average score for k={k}: {average_score}")


Average score for k=1: 13.84090909090909


KeyboardInterrupt: 